In [3]:
!pip list --not-required

Package         Version
--------------- -----------
parso           0.8.5
pillow          12.0.0
pip             24.3.1
platformdirs    4.5.0
prompt_toolkit  3.0.52
psutil          7.1.3
Pygments        2.19.2
pyparsing       3.2.5
python-dateutil 2.9.0.post0
pytz            2025.2
pyzmq           27.1.0
seaborn         0.13.2
stack-data      0.6.3
tornado         6.5.2
traitlets       5.14.3
tzdata          2025.2


# COVID-19 Data Analysis Project
Performs daily & weekly case calculation, rolling averages,
peak detection, and simple growth insight for selected countries
## Compatible with datasets from:
- Our World in Data (owid-covid-data.csv)
- WHO COVID dataset
- Kaggle COVID timeseries datasets

In [2]:
# Import section 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
# Load Dataset 
csv_file = r'src\datasets\'
if not os.path.exists(csv_file):
    raise FileNotFoundError(f"Dataset '{csv_file}' not found.")

df = pd.read_csv(csv_file, parse_dates=["date"])
print(f"Loaded dataset: {df.shape[0]} rows")
df.head(n= 10)

In [ ]:
# Filter countries & compute daily/weekly cases 
countries = ['','']
df = df[df["country"].isin(countries)].copy()

# Daily new cases if not available
if "new_cases" not in df.columns:
    df["new_cases"] = df.groupby("country")["cases"].diff().fillna(0)

# Weekly cases 
df["weekly_cases"] = df.groupby("country")["new_cases"].rolling(window=7).sum().reset_index(0, drop=True)
